# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# load the city.csv to perform the analysis
types_df = pd.read_csv("../output_data/city.csv")

# Drop any NA or unwanted data
types_df.dropna()
types = types_df
types.count()

Unnamed: 0         570
Country            565
City               570
Temperature        570
Max Temperature    570
Humidity           570
Cloud              570
Wind Speed         570
Latitude           570
Longtitude         570
dtype: int64

# Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Create configure gmaps plus API key
gmaps.configure(api_key=g_key)

# Create variables for locations and weight
locations = types[['Latitude', 'Longtitude']]
weight = types['Humidity'].astype(float)

In [4]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer to map
heat_layer = gmaps.heatmap_layer(locations, weight, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:

# Set zero cloudiness
zero_cloud = types[types["Cloud"] == 0]

# Set max temp above 70 degrees F
above_70_degrees = zero_cloud[zero_cloud["Max Temperature"] > 70]

# Set max temp below 80 degrees F
below_80_degrees = above_70_degrees[above_70_degrees["Max Temperature"] < 80]

# Set wind speed below 10 mph
slow_wind = below_80_degrees[below_80_degrees["Wind Speed"] < 10]

# Set humidity below 60 %
vacation_time = slow_wind[slow_wind["Humidity"] < 60]

# Set Index
indexed_vacation_time = vacation_time.reset_index()

del indexed_vacation_time["index"]

indexed_vacation_time


,Unnamed: 0,Country,City,Temperature,Max Temperature,Humidity,Cloud,Wind Speed,Latitude,Longtitude
0,15,CN,Dali,71.64,71.64,14,0,3.47,25.70,100.18
1,203,ZA,Port Shepstone,73.78,73.78,50,0,8.08,-30.74,30.45
2,538,AU,Batemans Bay,68.20,73.00,34,0,5.84,-35.72,150.18


In [6]:
vaca_locations = indexed_vacation_time[['Latitude', 'Longtitude']]
vaca_humidity = indexed_vacation_time['Humidity'].astype(float)

# Plot Heatmap
vaca_fig = gmaps.figure()

# Create heat layer
vaca_heat_layer = gmaps.heatmap_layer(vaca_locations, vaca_humidity, 
                                 dissipating=False, max_intensity=50,
                                 point_radius=2.5)

# Add layer
vaca_fig.add_layer(vaca_heat_layer)

# Display figure
vaca_fig

Figure(layout=FigureLayout(height='420px'))

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Need to work over this ... it is so much
# Hotel variable
hotels = []

# Loop through dataframe to get nearest hotel
for city in range(len(indexed_vacation_time["City"])):

    lat = indexed_vacation_time.loc[city]["Latitude"]
    lng = indexed_vacation_time.loc[city]["Longtitude"]

    city_coords = f"{lat},{lng}"

    params = {
        "location": city_coords, 
        "types": "lodging",
        "radius": 5000,
        "key": g_key
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   

    hotel_request = requests.get(base_url, params=params)
    hotel_response = hotel_request.json()

    try:
        hotels.append(hotel_response["results"][0]["name"])
    except:
        hotels.append("Nearest hotel not found")

# Dataframe with nearest hotel
indexed_vacation_time["Nearest Hotel"] = hotels
indexed_vacation_time

,Unnamed: 0,Country,City,Temperature,Max Temperature,Humidity,Cloud,Wind Speed,Latitude,Longtitude,Nearest Hotel
0,15,CN,Dali,71.64,71.64,14,0,3.47,25.70,100.18,Dali Travelling With Hostel
1,203,ZA,Port Shepstone,73.78,73.78,50,0,8.08,-30.74,30.45,My Den Beachfront B&B And Self-Catering
2,538,AU,Batemans Bay,68.20,73.00,34,0,5.84,-35.72,150.18,Coachhouse Marina Resort - Batemans Bay


In [8]:
# NOTE: Do not change any of the code in this cell
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Nearest Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in indexed_vacation_time.iterrows()]
locations = indexed_vacation_time[["Latitude", "Longtitude"]]
locations

,Latitude,Longtitude
0,25.70,100.18
1,-30.74,30.45
2,-35.72,150.18


In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content= [f"Nearest Hotel: {hotel}" for hotel in hotels])
vaca_fig.add_layer(markers)

# Display Map
vaca_fig

Figure(layout=FigureLayout(height='420px'))